In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/dhamvi01/Univariate-Time-Series-using-LSTM/master/airline-passengers.csv'
data = pd.read_csv(url)
data

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [2]:
passengers = data['Passengers'].values
passengers

array([112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118, 115,
       126, 141, 135, 125, 149, 170, 170, 158, 133, 114, 140, 145, 150,
       178, 163, 172, 178, 199, 199, 184, 162, 146, 166, 171, 180, 193,
       181, 183, 218, 230, 242, 209, 191, 172, 194, 196, 196, 236, 235,
       229, 243, 264, 272, 237, 211, 180, 201, 204, 188, 235, 227, 234,
       264, 302, 293, 259, 229, 203, 229, 242, 233, 267, 269, 270, 315,
       364, 347, 312, 274, 237, 278, 284, 277, 317, 313, 318, 374, 413,
       405, 355, 306, 271, 306, 315, 301, 356, 348, 355, 422, 465, 467,
       404, 347, 305, 336, 340, 318, 362, 348, 363, 435, 491, 505, 404,
       359, 310, 337, 360, 342, 406, 396, 420, 472, 548, 559, 463, 407,
       362, 405, 417, 391, 419, 461, 472, 535, 622, 606, 508, 461, 390,
       432])

In [3]:
# transform univariate time series to supervised learning problem
from numpy import array
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

# define univariate time series
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# transform to a supervised learning problem
X, y = split_sequence(passengers, 2)
print(X.shape, y.shape)
shapeX = X.shape

dfX = pd.DataFrame(X, columns=["Xt-2","Xt-1"])
dfy = pd.DataFrame(y, columns=["Xt"])

df = pd.concat((dfX, dfy), axis = 1)
df

(142, 2) (142,)


,Xt-2,Xt-1,Xt
0,112,118,132
1,118,132,129
2,132,129,121
3,129,121,135
4,121,135,148
...,...,...,...
137,535,622,606
138,622,606,508
139,606,508,461
140,508,461,390


In [4]:
from sklearn.preprocessing import MinMaxScaler
import joblib

scaler = MinMaxScaler()
#scaler.fit(features)
#scaler.transform(features)
scaled = scaler.fit_transform(df)
features_names = df.columns.copy()
#features_names.remove('label')
scaled_features = pd.DataFrame(scaled, columns=features_names)
scaled_features

,Xt-2,Xt-1,Xt
0,0.015444,0.027027,0.054054
1,0.027027,0.054054,0.048263
2,0.054054,0.048263,0.032819
3,0.048263,0.032819,0.059846
4,0.032819,0.059846,0.084942
...,...,...,...
137,0.832046,1.000000,0.969112
138,1.000000,0.969112,0.779923
139,0.969112,0.779923,0.689189
140,0.779923,0.689189,0.552124


In [5]:
dataX = scaled_features.drop(columns=['Xt'])
dataX

,Xt-2,Xt-1
0,0.015444,0.027027
1,0.027027,0.054054
2,0.054054,0.048263
3,0.048263,0.032819
4,0.032819,0.059846
...,...,...
137,0.832046,1.000000
138,1.000000,0.969112
139,0.969112,0.779923
140,0.779923,0.689189


In [6]:
dataY = scaled_features['Xt'].values
dataY

array([0.05405405, 0.04826255, 0.03281853, 0.05984556, 0.08494208,
       0.08494208, 0.06177606, 0.02895753, 0.        , 0.02702703,
       0.02123552, 0.04247104, 0.07142857, 0.05984556, 0.04054054,
       0.08687259, 0.12741313, 0.12741313, 0.1042471 , 0.05598456,
       0.01930502, 0.06949807, 0.07915058, 0.08880309, 0.14285714,
       0.11389961, 0.13127413, 0.14285714, 0.18339768, 0.18339768,
       0.15444015, 0.11196911, 0.08108108, 0.11969112, 0.12934363,
       0.14671815, 0.17181467, 0.14864865, 0.15250965, 0.22007722,
       0.24324324, 0.26640927, 0.2027027 , 0.16795367, 0.13127413,
       0.17374517, 0.17760618, 0.17760618, 0.25482625, 0.25289575,
       0.24131274, 0.26833977, 0.30888031, 0.32432432, 0.25675676,
       0.20656371, 0.14671815, 0.18725869, 0.19305019, 0.16216216,
       0.25289575, 0.23745174, 0.25096525, 0.30888031, 0.38223938,
       0.36486486, 0.2992278 , 0.24131274, 0.19111969, 0.24131274,
       0.26640927, 0.24903475, 0.31467181, 0.31853282, 0.32046

In [7]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(dataX,test_size=0.2, random_state=1)#Nilai X training dan Nilai X testing
training_label, test_label = train_test_split(dataY, test_size=0.2, random_state=1)#Nilai Y training dan Nilai Y testing

In [9]:
from sklearn.neighbors import NearestNeighbors
regresor = NearestNeighbors(n_neighbors=3)
regresor.fit(training)

NearestNeighbors(n_neighbors=3)

In [11]:
from sklearn.metrics import accuracy_score

# Membuat prediksi menggunakan data testing
test_pred = regresor.predict(test)

# Menghitung akurasi
accuracy = accuracy_score(test_label, test_pred)

print("Akurasi KNN: {:.2f}%".format(accuracy * 100))


AttributeError: ignored